<a href="https://colab.research.google.com/github/AshwinKotgire/Play_with_LLMs/blob/main/Prompt_Engineering/Rough_script_of_LLM_with_context.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U trl transformers accelerate
!pip install -q datasets bitsandbytes einops wandb
!pip install langchain
!pip install sentence_transformers
# !pip install faiss-cpu
!pip install faiss-gpu
!pip install pypdf
!pip install  tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from huggingface_hub import login
login()

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from transformers import pipeline
import transformers
import torch


In [3]:

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import UnstructuredURLLoader
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.document_loaders import PDFMinerPDFasHTMLLoader
from langchain.document_loaders import PDFPlumberLoader
from langchain.text_splitter import TokenTextSplitter

from IPython.display import HTML

from langchain.document_loaders import OnlinePDFLoader



In [7]:

model_name = "meta-llama/Llama-2-7b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
from IPython.display import Markdown
if torch.cuda.is_available():
    print('cuda')
    device = torch.device("cuda")  # If GPU is available, use it.
else:
    device = torch.device("cpu")   # If GPU is not available, use the CPU.



cuda


In [4]:
urls="https://arxiv.org/pdf/1706.03762.pdf"
loader=PyPDFLoader(urls)
data=loader.load()

In [8]:
content=' '
for i in data : content+=i.page_content
content

' Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and co

In [9]:
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=200)

texts = text_splitter.create_documents([content])


In [10]:
embeddings = HuggingFaceEmbeddings()

In [11]:
db = FAISS.from_documents(texts, embeddings)

In [38]:
question="What is the difference between rnn and transformers?"
contexts=db.similarity_search(question,k=1)
print(contexts)

[Document(page_content=' 3 rows (B), we observe that reducing the attention key size dkhurts model quality. This\nsuggests that determining compatibility is not easy and that a more sophisticated compatibility\nfunction than dot product may be beneficial. We further observe in rows (C) and (D) that, as expected,\nbigger models are better, and dropout is very helpful in avoiding over-fitting. In row (E) we replace our\nsinusoidal positional encoding with learned positional embeddings [ 9], and observe nearly identical\nresults to the base model.\n6.3 English Constituency Parsing\nTo evaluate if the Transformer can generalize to other tasks we performed experiments on English\nconstituency parsing. This task presents specific challenges: the output is subject to strong structural\nconstraints and is significantly longer than the input. Furthermore, RNN sequence-to-sequence\nmodels have not been able to attain state-of-the-art results in small-data regimes [37].\nWe trained a 4-layer tran

In [39]:
print(len(contexts))

1


In [40]:
context=' '
for i in contexts:context+=i.page_content
context

'  3 rows (B), we observe that reducing the attention key size dkhurts model quality. This\nsuggests that determining compatibility is not easy and that a more sophisticated compatibility\nfunction than dot product may be beneficial. We further observe in rows (C) and (D) that, as expected,\nbigger models are better, and dropout is very helpful in avoiding over-fitting. In row (E) we replace our\nsinusoidal positional encoding with learned positional embeddings [ 9], and observe nearly identical\nresults to the base model.\n6.3 English Constituency Parsing\nTo evaluate if the Transformer can generalize to other tasks we performed experiments on English\nconstituency parsing. This task presents specific challenges: the output is subject to strong structural\nconstraints and is significantly longer than the input. Furthermore, RNN sequence-to-sequence\nmodels have not been able to attain state-of-the-art results in small-data regimes [37].\nWe trained a 4-layer transformer with dmodel = 

In [41]:
prompt=f"""<s>[INST] <<SYS>>
You are an honest assistant , who gives factually correct answers while refering to the context. If the context does not have an answer you try your best to answer the question on your own.\n
Answer the following question while refering the context.If the answer is not in context ,try to answer on your own . Still if you are not able to answer the question then politely say so instead of ramblling.\n
<</SYS>>
Context:{context}
Qustion:{question}
 [/INST]
"""
print(prompt)

<s>[INST] <<SYS>>
You are an honest assistant , who gives factually correct answers while refering to the context. If the context does not have an answer you try your best to answer the question on your own.

Answer the following question while refering the context.If the answer is not in context ,try to answer on your own . Still if you are not able to answer the question then politely say so instead of ramblling.

<</SYS>>
Context:  3 rows (B), we observe that reducing the attention key size dkhurts model quality. This
suggests that determining compatibility is not easy and that a more sophisticated compatibility
function than dot product may be beneficial. We further observe in rows (C) and (D) that, as expected,
bigger models are better, and dropout is very helpful in avoiding over-fitting. In row (E) we replace our
sinusoidal positional encoding with learned positional embeddings [ 9], and observe nearly identical
results to the base model.
6.3 English Constituency Parsing
To eval

In [15]:
from IPython.display import Markdown

In [42]:
# prompt = 'I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?'
inputs = tokenizer(prompt, return_tensors="pt",padding =True).to(device)
generate_ids = model.generate(inputs.input_ids, max_length=5000,top_k=1,top_p=0.5,temperature=0)


In [67]:
p=generate_ids[0][inputs['input_ids'].shape[1]:]

In [68]:
p

tensor([ 1576,  4328,  1546,   390, 10262, 29879,   322,  4103,   689,   414,
          338,   393,   390, 10262, 29879,   671,  1162,  1264, 12368,   304,
         1889, 15602, 29892,  1550,  4103,   689,   414,   671,  1583, 29899,
         1131,  2509,  7208, 12903,   304,  1889, 15602, 29889,    13, 29934,
        10262, 29879,   313,  4789,  1264,  2448,  3631,  8527, 29879, 29897,
          526,   263,  1134,   310, 19677,  3564, 11258,   393,   338, 10734,
         1532, 29899,  2146,  1573,   363,  9068,  8617,  2556,   848, 29892,
         1316,   408,   931,  3652,   848,   470,  5613,  4086,  1426, 29889,
          390, 10262, 29879,   671,  1162,  1264, 12368,   304,  7344,   263,
         7934,  2106,   393,  4332,  1973,  2472,   515,  3517,   931,  6576,
        29892, 14372,   963,   304,  1207, 27303,  2729,   373,   445,  2472,
        29889,    13, 13372,   414, 29892,   373,   278,   916,  1361, 29892,
          526,   263,   901,  7786,  1134,   310, 19677,  3564, 

In [69]:

Markdown("###"+tokenizer.decode(p, skip_special_tokens=True, clean_up_tokenization_spaces=True))
# print(tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0])

###The difference between RNNs and Transformers is that RNNs use recurrent connections to process sequences, while Transformers use self-attention mechanisms to process sequences.
RNNs (Recurrent Neural Networks) are a type of neural network architecture that is particularly well-suited for processing sequential data, such as time series data or natural language text. RNNs use recurrent connections to maintain a hidden state that captures information from previous time steps, allowing them to make predictions based on this information.
Transformers, on the other hand, are a more recent type of neural network architecture that was introduced in the paper "Attention is All You Need" by Vaswani et al. in 2017. Transformers are designed primarily for natural language processing tasks, such as machine translation and language modeling. Transformers use self-attention mechanisms to process sequences, which allows them to parallelize the computation of attention across different parts of the input sequence. This makes them much faster and more scalable than RNNs for long sequences.
Here are some key differences between RNNs and Transformers:
1. Processing mechanism: RNNs use recurrent connections to process sequences, while Transformers use self-attention mechanisms.
2. Sequence length: RNNs are limited by the length of the sequence they are processing, while Transformers can handle sequences of arbitrary length.
3. Computational complexity: RNNs have a computational complexity of O(n^2), where n is the length of the sequence, while Transformers have a computational complexity of O(n), making them much faster for long sequences.
4. Training time: RNNs typically require more training data and time than Transformers, due to their recurrent connections and sequential processing mechanism.
5. Parallelization: Transformers can be parallelized more easily than RNNs, as the self-attention mechanism allows for parallel computation across different parts of the input sequence.
6. Performance: Transformers have been shown to perform better than RNNs on many natural language processing tasks, such as machine translation and language modeling.
In summary, RNNs and Transformers are both powerful tools for processing sequential data, but they have different strengths and weaknesses. RNNs are better suited for tasks that require sequential processing and maintenance of a hidden state, while Transformers are better suited for tasks that require parallelization and can handle sequences of arbitrary length.

In [48]:
a

'[INST] <<SYS>>\nYou are an honest assistant , who gives factually correct answers while refering to the context. If the context does not have an answer you try your best to answer the question on your own.\n\nAnswer the following question while refering the context.If the answer is not in context ,try to answer on your own . Still if you are not able to answer the question then politely say so instead of ramblling.\n\n<</SYS>>\nContext:  3 rows (B), we observe that reducing the attention key size dkhurts model quality. This\nsuggests that determining compatibility is not easy and that a more sophisticated compatibility\nfunction than dot product may be beneficial. We further observe in rows (C) and (D) that, as expected,\nbigger models are better, and dropout is very helpful in avoiding over-fitting. In row (E) we replace our\nsinusoidal positional encoding with learned positional embeddings [ 9], and observe nearly identical\nresults to the base model.\n6.3 English Constituency Parsi